In [1]:
%pip install openai
%pip install pandas
%pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 977.7 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 814.4 kB/s eta 0:00:00:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 12.6 MB/s eta 0:00:0000:01:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 16.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 7.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 12.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 630.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 16.7 MB/s eta 0:00:000

In [40]:
from openai import OpenAI
import re
import numpy as np
import pandas as pd
import json
import os

In [41]:
# set your openai api key here
os.environ['OPENAI_API_KEY']=''

In [42]:
datagen_model = "gpt-4o"
client = OpenAI()

In [25]:
examples = [
    {
        "description": "Simple RC circuit with a resistor and capacitor connected in series.",
        "spice": """
* Simple RC circuit
V1 1 0 DC 10
R1 1 2 1k
C1 2 0 1u
.tran 1
.end
""",
        "voltages": {
            "1": 10,
            "2": 9.9
        }
    }
]

In [26]:
def generate_prompt(examples):
    prompt = "Generate synthetic data for circuits and chip design. Each data should contain:\n1. A human description of a circuit.\n2. The circuit presented in SPICE language.\n3. Voltages of each node in the circuit.\n\nExamples:\n"
    for example in examples:
        prompt += f"Description: {example['description']}\nSPICE:\n{example['spice']}\nVoltages: {json.dumps(example['voltages'], indent=4)}\n\n"
    prompt += "Generate a new synthetic data example:\nDescription: "
    return prompt

In [35]:
def generate_synthetic_data():
    prompt = generate_prompt(examples)
    response = client.chat.completions.create(
        model=datagen_model,
        messages=[
          {"role": "system", "content": "You are a helpful assistant designed to generate synthetic data."},
          {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        n=1,
        stop=None,
        temperature=0.7
    )
    
    res = response.choices[0].message.content
    return res

In [37]:
for _ in range(3):
    synthetic_data = generate_synthetic_data()
    print("Generated Synthetic Data:")
    print(synthetic_data)

    # Run the SPICE code through simulator to verify the voltages.
    # verified_synthetic_data = { "description": ..., "spice": ..., "voltages": ... } (parse the generated data accordingly)
    # examples.append(verified_synthetic_data)


    # design metrics to measure how
    #     rate of good ones

Generated Synthetic Data:
Description: Voltage divider circuit with two resistors connected in series.

SPICE:

* Voltage divider circuit
V1 1 0 DC 12
R1 1 2 2k
R2 2 0 1k
.tran 1
.end

Voltages: {
    "1": 12,
    "2": 4
}
Generated Synthetic Data:
Description: A basic RL circuit consisting of a resistor and an inductor connected in series, powered by a DC voltage source.

SPICE:

* Basic RL circuit
V1 1 0 DC 12
R1 1 2 2k
L1 2 0 10m
.tran 1
.end

Voltages: {
    "1": 12,
    "2": 12
}
Generated Synthetic Data:
Description: A basic RL circuit with a resistor and an inductor connected in series.

SPICE:

* Basic RL circuit
V1 1 0 DC 5
R1 1 2 500
L1 2 0 1m
.tran 1
.end

Voltages: {
    "1": 5,
    "2": 4.5
}
